In [1]:
import pandas as pd
import numpy as np

In [2]:
db = pd.read_excel('dataset.xlsx')
db = db[db.columns.drop(list(db.filter(regex='^Unnamed')))]

# Есть возможность преобразовать какие-то столбцы с данными в уникальные идентификаторы, чтобы не хранить избыточную информацию. Также создать таблицу справочник для сопоставления id к информации. 

## Для начала преобразуем department в id и создадим таблицу справочник

In [3]:
unique_deps = list(set(db['department']))
deps_ds = pd.DataFrame(list(zip(range(len(unique_deps)), unique_deps)), columns =['dep_id', 'department'])
db = pd.merge(db, deps_ds, how='inner', on = 'department')

## Аналогично можно с hometown, однако там нет унифицированной информации. Данные об одном месте могут называться по-разному. Если и обрабатывать данные, то явно другим способом. Пример разных названий:

In [4]:
print(db['hometown'][0])
print(db['hometown'][1])
print(db['hometown'][26528])

 ГОР. КАЗАНЬ
 РОССИЯ ГОР. КАЗАНЬ
 423057, РОССИЯ, РЕСП. ТАТАРСТАН (ТАТАРСТАН), АКСУБАЕВСКИЙ Р-Н, С. НОВОЕ ДЕМКИНО


## Изменим дату рождения. Есть строки, которые не содержат дат, нужно их почистить и оставить только подходящие.

In [5]:
db['birthday']=db['birthday'].str.replace('.','')

In [6]:
db['birthday'] = db['birthday'].astype(str)
db['birthday_digit'] = list(map(lambda x: x.isdigit(), db['birthday']))

In [7]:
print('Количество строк с цифрами {}. Количество строк без цифр {}'
      .format(len(db[db['birthday_digit'] == True]), len(db[db['birthday_digit'] == False])))

Количество строк с цифрами 26441. Количество строк без цифр 92


In [8]:
db['birthday'] = db[db['birthday_digit'] == True]['birthday']
db['birthday'] = pd.to_datetime(db['birthday'], format='%d%m%Y')

## number_ip можно разбить на уникальный идентификатор и дату
## из поля details можно выявить какие-то уникальные слова
## из поля credit можно получить тип задолжености и сумму исполнительного сбора

In [9]:
db['credit'] = db['credit'].astype(str)
for i in range(len(db)):
    # получение id, даты и года исполнительного производства
    arr = db['number_ip'][i].split(' ')
    db.loc[i, 'number_ip_id'] = arr[0]
    db.loc[i, 'number_ip_date'] = arr[2]
    arr = db['number_ip_date'][i].split('.')
    db.loc[i, 'year'] = arr[2]
    
    # получение уникальных значений описания "на коленке)"
    splt = db['details'][i].split(' ')
    db.loc[i, 'uniq_details'] = splt[0] + ' ' + splt[1]
    
    # получение уникальных значений по типу задолженности    
    arr = db['credit'][i].split(': ')
    db.loc[i, 'credit_type'] = arr[0]
    if len(arr) > 1:
        if ('Исполнительский сбор' in arr[1]):
            db.loc[i, 'sum_rub'] = arr[1].split(' ')[0]
            if len(arr) == 3:
                db.loc[i, 'ip_sum'] = arr[2].split()[0]

## Преобразуем поле статус в типы статуса, для этого проделаем некоторые преобразования по приведению к стандартному виду и созданию типов на этих стандартах

In [10]:
print('как выглядело до')
db['status']

как выглядело до


0        2020-03-18, 46, 1, 3
1                         NaN
2        2020-10-19, 46, 1, 3
3        2020-01-20, 46, 1, 3
4        2019-01-17, 46, 1, 3
                 ...         
26528                     NaN
26529                     NaN
26530    2018-05-17, 46, 1, 3
26531                     NaN
26532                     NaN
Name: status, Length: 26533, dtype: object

In [11]:
db['status'] = db['status'].str[12:] 
db['status'] = db['status'].replace(np.nan, '0')
db['status'] = db['status'].replace('', '0')
print('как выглядело после')
db['status']

как выглядело после


0        46, 1, 3
1               0
2        46, 1, 3
3        46, 1, 3
4        46, 1, 3
           ...   
26528           0
26529           0
26530    46, 1, 3
26531           0
26532           0
Name: status, Length: 26533, dtype: object

In [12]:
# стандартизируем и джойним к основной таблице
unique_status = list(set(db['status']))
status_ds = pd.DataFrame(list(zip(range(len(unique_status)), unique_status)), columns =['status_id', 'status'])
db = pd.merge(db, status_ds, how='inner', on = 'status')
db

,name,lastname,secondname,birthday,hometown,number_ip,credit,details,department,status,dep_id,birthday_digit,number_ip_id,number_ip_date,year,uniq_details,credit_type,sum_rub,ip_sum,status_id
0,ЕГОР,РОГОВ,НИКОЛАЕВИЧ,1985-07-31,ГОР. КАЗАНЬ,129954/19/16007-ИП от 23.12.2019,nan,Постановление судебного пристава-исполнителя о...,Приволжское РОСП г.Казани УФССП России по Респ...,"46, 1, 3",45,True,129954/19/16007-ИП,23.12.2019,2019,Постановление судебного,nan,NaN,NaN,4
1,РАМИЛЬ,ГАЙСИН,РОБЕРТОВИЧ,1993-05-19,Г. КАЗАНЬ,9013/20/16007-ИП от 03.02.2020,nan,Постановление судебного пристава-исполнителя о...,Приволжское РОСП г.Казани УФССП России по Респ...,"46, 1, 3",45,True,9013/20/16007-ИП,03.02.2020,2020,Постановление судебного,nan,NaN,NaN,4
2,ГУЗЕЛЬ,ГАЛИМОВА,КАМИЛЕВНА,1990-09-01,Г. КАЗАНЬ,60056/17/16007-ИП от 03.10.2016,Задолженность,Судебный приказ от 20.11.2015 № 2-1756/15 СУДЕ...,Приволжское РОСП г.Казани УФССП России по Респ...,"46, 1, 3",45,True,60056/17/16007-ИП,03.10.2016,2016,Судебный приказ,Задолженность,NaN,NaN,4
3,ГУЗЕЛЬ,ГАЛИМОВА,КАМИЛЕВНА,1990-09-01,Г. КАЗАНЬ,27869/18/16007-ИП от 04.05.2018,Задолженность по кредитным платежам (кроме ипо...,Исполнительный лист от 14.12.2016 № 2-8350/201...,Приволжское РОСП г.Казани УФССП России по Респ...,"46, 1, 3",45,True,27869/18/16007-ИП,04.05.2018,2018,Исполнительный лист,Задолженность по кредитным платежам (кроме ипо...,NaN,NaN,4
4,ГУЗЕЛЬ,ГАЛИМОВА,КАМИЛЕВНА,1990-09-01,Г. КАЗАНЬ,65669/19/16007-ИП от 05.08.2019,Задолженность по кредитным платежам (кроме ипо...,Исполнительный лист от 14.12.2016 № 2-8350/201...,Приволжское РОСП г.Казани УФССП России по Респ...,"46, 1, 3",45,True,65669/19/16007-ИП,05.08.2019,2019,Исполнительный лист,Задолженность по кредитным платежам (кроме ипо...,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26528,МАРАТ,ГАЛЯУТДИНОВ,РЕНАТОВИЧ,1992-01-24,"РЕСП. ТАТАРСТАН, Г. АРСК",128192/18/16018-ИП от 18.12.2018,Задолженность по кредитным платежам (кроме ипо...,Судебный приказ от 05.10.2018 № 2-1102/1/2018 ...,ОСП по Арскому и Атнинскому районам УФССП Росс...,"46, 1, 4",34,True,128192/18/16018-ИП,18.12.2018,2018,Судебный приказ,Задолженность по кредитным платежам (кроме ипо...,NaN,NaN,2
26529,ИГОРЬ,МОРОЗОВ,АЛЕКСАНДРОВИЧ,1983-09-28,"РОССИЯ, РЕСП. ТАТАРСТАН (ТАТАРСТАН), АКСУБАЕВ...",13754/14/16012-ИП от 21.11.2014,nan,Постановление судебного пристава-исполнителя о...,Аксубаевское РОСП УФССП России по Республике Т...,"46, 1, 4",37,True,13754/14/16012-ИП,21.11.2014,2014,Постановление судебного,nan,NaN,NaN,2
26530,ТАТЬЯНА,ПЕТРОВА,ГЕННАДЬЕВНА,1985-01-12,"САМАРСКАЯ ОБЛ., С. БОЛЬШАЯ ЧЕРНИГОВКА",1333/18/16012-ИП от 01.02.2018,"Госпошлина, присужденная судом",Исполнительный лист от 21.04.2017 № 2-19/2017 ...,Аксубаевское РОСП УФССП России по Республике Т...,"46, 1, 4",37,True,1333/18/16012-ИП,01.02.2018,2018,Исполнительный лист,"Госпошлина, присужденная судом",NaN,NaN,2
26531,ЛЕНАР,МУРТАЗИН,РИНАТОВИЧ,1968-03-30,ГОР. КАЗАНЬ,135288/19/16006-ИП от 26.09.2018,Задолженность по платежам за услуги связи,Судебный приказ от 20.11.2017 № 2-1283/17 СУДЕ...,Ново-Савиновский РОСП г.Казани УФССП России по...,"47, 1, 6",36,True,135288/19/16006-ИП,26.09.2018,2018,Судебный приказ,Задолженность по платежам за услуги связи,NaN,NaN,3


In [13]:
# get finish datasets : 
# FINISH TABLE for ANALYSIS: number_ip_id, number_ip_date, year, birthday, hometown, dep_id,uniq_details, credit_type,sum_rub,ip_sum,status_id
finish_table = pd.DataFrame({
    'number_ip_id':db['number_ip_id'], 
    'number_ip_date': db['number_ip_date'],
    'year': db['year'],
    'birthday': db['birthday'],
    'hometown': db['hometown'],
    'dep_id': db['dep_id'],
    'uniq_details': db['uniq_details'],
    'credit_type': db['credit_type'],
    'sum_rub': db['sum_rub'],
    'ip_sum': db['ip_sum'],
    'status_id': db['status_id'],
    })
finish_table.to_excel('finish_table.xlsx')

# DIM STATUS: status_id, status
status_ds.to_excel('dim_status.xlsx')

# DIM DEPARTMENT: dep_id, department
deps_ds.to_excel('dim_department.xlsx')

#DIM IP
dim_ip = pd.DataFrame({
    'number_ip_id':db['number_ip_id'], 
    'number_ip': db['number_ip'],
    'name': db['name'],
    'lastname': db['lastname'],
    'secondname': db['secondname'],
    'credit': db['credit'],
    'details': db['details'],
    })
dim_ip.to_excel('dim_ip.xlsx')